In [1]:
import os
import numpy as np
import pandas as pd
import itertools
import codecs
from full_half_conversion import *

from datetime import datetime
from datetime import timedelta as td

ModuleNotFoundError: No module named 'full_half_conversion'

■■■■■　関数　■■■■■

In [2]:
def GetCSVFiles():
    files = os.listdir(r_path)
    csv_files = []
    
    for t in  files:
        if(t[-4:] == '.csv' and (t[:2].upper() == "NG" or t[:2].upper() == "OK")):
             csv_files.append(t)
    
    return csv_files

In [3]:
def GetSAExtractedFiles():
    files = os.listdir(w_path)
    csv_files = []
    
    for file in files:
        if(file.find(".csv") != -1 and file.startswith("SA_Sub_")):
             csv_files.append(file)
    
    return csv_files

In [4]:
def GetAssyExtractedFiles():
    files = os.listdir(w_path)
    csv_files = []
    
    for file in files:
        if(file.find(".csv") != -1 and file.startswith("Assy_Sub_")):
             csv_files.append(file)
    
    return csv_files

In [47]:
def GetAssyWithGelExtractedFiles():
    files = os.listdir(w_path)
    csv_files = []
    
    for file in files:
        if(file.find(".csv") != -1 and file.startswith("AssyWithGel_")):
             csv_files.append(file)
    
    return csv_files

In [5]:
def GetAllExtractedFiles():
    files = os.listdir(w_path)
    csv_files = []
    
    for file in files:
        if(file.find(".csv") != -1 and file.find("Sub_") != -1):
             csv_files.append(file)
    
    return csv_files

In [6]:
def CreateSADefectiveRatio():
    
    #不良率ファイル読み込み
    dr_path = os.path.join("..", "＜更新＞サブアッシー不良率.xlsx")    
    df = pd.read_excel(dr_path, encoding="SHIFT-JIS", skiprows=[0,1], usecols=[4,5,6,7,9,10,11,12], dtype = {"製番":"object"}, sheet_name=0)
        
    #列名をリネーム
    new_cols = ["Lot", "WorkNum", "DefectNum", "DefectRatio", "sub1", "sub2", "sub3", "sub4"]
    df.columns = new_cols
    
    #いらんとこドロップ
    ks = df.apply(lambda s: s.astype(str).str.contains("要確認", na=False))
    df = df.mask(ks)
    df = df.dropna(subset=["WorkNum", "DefectNum", "sub1"])
    df = df.drop_duplicates()
    
    df = df.reset_index()
    df = df.drop("index", axis=1)
    
    #全角を半角に
    for i in np.arange(df.shape[0]):
        for j in [1,2,3,4]:
            if not pd.isnull(df.loc[i, "sub%s" %j]):
                df.loc[i, "sub%s" %j] = zen2han(df.loc[i, "sub%s" %j])
    
    #その他
    #本当は，./で区切られて1セルに複数ロットあるのとか，E07xxのような文字列入ってるのとかあるけど，今回のロットには関係ないので省略
    #これは入力側で頑張ってもらいたい．めんどい
    
    #書き出し
    df.to_csv(os.path.join("..", "SA_DefectiveRatio.csv"), index=False, encoding="utf-8")

In [7]:
#未完成　やってられん
def CreateSubstrateSpecs():
    dr_path = os.path.join("..", "AssemblyData", "Part_Spec", "substrate_spec.xlsx")
    re_cols = {"京セラ\n出荷日":"delivery_date", "京セラ手配No":"logis_no", "出荷\nﾛｯﾄNo":"lot_no", "セラミック\nロット":"ceramic_lot",
               "寸法\nランク":"sz_rank", "1次+2次Niめっき厚み[μm]":"Ni_thick[um]", "Pdめっき厚み[μm]":"Pd_thick[um]",
               "Auめっき厚み[μm]":"Au_thick[um]", "Ni/Pd/Auめっき":"Ni/Pd/Au_plating", "Pdめっき":"Pd_plating", "Auめっき":"Au_plating",
               "電気伝導度（水洗）[μS/cm]":"elec_conduct_cw[uS/cm]", "電気伝導度（湯煎）[μS/cm]":"elec_conduct_hw[uS/cm]",
               ""}
    
    
    temp = pd.read_excel(dr_path, encoding="SHIFT-JIS", skiprows=7, header=None, sheet_name=0)
    temp = temp.drop(0, axis=1)
    header = temp[:2]
    df = temp[2:]
    
    
    
    header = header.fillna(method="ffill", axis=1)
    
    df = df.fillna(method="ffill")
    
    print(header.head())
    print(df.head())
    
#CreateSubstrateSpecs()    

SyntaxError: invalid syntax (<ipython-input-7-1559c8b0715c>, line 7)

In [8]:
#時間差をsecで出力
def CalcDeltaSeconds(t):
    t1, t2 = t
    delta = t2 - t1
    
    return delta.total_seconds()

In [9]:
#1セル内の,で区切られたデータの分割
#データの並び的に，前側のデータを使えば良さそうだったので，前側のみ抽出
def DecomposeStr(x):
    if type(x) in (str, object):
        point = x.find(",")
        
        if point != -1:
            x = x[:point]
            
    return x

In [45]:
#16進数を10進数に
def HEX2DN(x):
    
    if x != x:  #NANの場合
        return x
    else:
        x = str(x)
        
        point = x.find(".")
        if point != -1:
            x = x[:point]
        
        return int(x, 16)

In [11]:
#SA組付工程
def CreateSABaseData(csv_file):
    
    #工程番号
    pro_no = np.arange(11, 17)
    
    #読み込む列
    #"chip_id","assy_lot", "assy_hinban",は16番でNGのものはNAになっているので，省いた
    col_output  = ["out_put_time_%s"%i for i in pro_no]
    col_others  = ["sa_lot", "sa_seri", "sa_hinban", "kiban_lot", "film_lot",
                   "格納位置_14", "格納時間_14", "排出時間_14", "不良カテゴリ_16"]
    col_rename  = {"格納位置_14": "storage_loc_14",
                   "格納時間_14": "storage_start_14",
                   "排出時間_14": "storage_end_14",
                   "不良カテゴリ_16": "defective_cat_16"}
    col_process = col_others + col_output
    
    #指定工程Noのデータ読み込み
    with codecs.open(os.path.join(r_path, csv_file), "r", "Shift-JIS", "ignore") as f:
        reader = pd.read_table(f, delimiter=",", low_memory=False, usecols=col_process, chunksize=1000)
        df = pd.concat((r for r in reader), ignore_index=True)

    #列名をリネーム
    df = df.rename(columns=col_rename)
    
    #日付：オブジェクト　→　datetimeへ変換
    for i in pro_no:
        df["out_put_time_%s" %i] = pd.to_datetime(df["out_put_time_%s" %i])
        
    df["storage_start_14"] = pd.to_datetime(df["storage_start_14"])
    df["storage_end_14"]   = pd.to_datetime(df["storage_end_14"])
    
    #工程間の時間　秒
    for i in pro_no[:-1]:
        df["ti_%s_to_%s[s]" %(i, i+1)] = df[["out_put_time_%s" %(i), "out_put_time_%s" %(i+1)]].apply(CalcDeltaSeconds, axis=1)
    
    #マンション在宅時間　秒
    df["cure_time[s]"] = df[["storage_start_14", "storage_end_14"]].apply(CalcDeltaSeconds, axis=1)
    
    #基板ロットが1セルに複数入っているのを分割
    df["kiban_lot"] = df["kiban_lot"].map(DecomposeStr)
    
    #先にドロップすると，色んな行のNAにヒットしてデータがほぼなくなるので注意
    df = df.dropna()
    df = df.reset_index()
    df = df.drop("index", axis=1)
    
    #不良ファイルから，基板品番→基板ロット情報抽出
    #定義＆読み込み
    sub_cols=["sub%s" %i for i in [1,2,3,4]]
    df_sub = pd.read_csv(os.path.join("..", "SA_DefectiveRatio.csv"))
    df_sub_lot = pd.DataFrame(index=[], columns=sub_cols)
    print(df_sub.info())
    #1行ずつ，ヒットする基板Noを抽出
    for i in np.arange(df.shape[0]):
        
        sub_lot = df_sub[df_sub["Lot"].astype(str) == df.loc[i, "kiban_lot"]]

        if len(sub_lot) == 0:
            sub_ser = pd.Series([np.nan, np.nan, np.nan, np.nan], index=df_sub_lot.columns)
        else:
            sub_ser = sub_lot.loc[:, sub_cols]
        df_sub_lot = df_sub_lot.append(sub_ser, ignore_index=True)

    #大元のファイルと結合
    df = pd.concat([df, df_sub_lot], axis=1)
   
    #書き出し
    df.to_csv(os.path.join(w_path, "SA_Sub_" + csv_file), index=False, encoding="utf-8")

In [12]:
#Assy基板接着
def CreateAssyBaseData(csv_file):
    
    #工程番号
    pro_no = np.arange(32, 34)
    
    #読み込む列
    #"chip_id","assy_lot", "assy_hinban",は16番でNGのものはNAになっているので，省いた
    col_out  = ["out_put_time_16"]+["out_put_time_%s"%i for i in pro_no]
    col_others  = ["sa_lot", "sa_seri", "sa_hinban", "kiban_lot", "film_lot", "assy_lot", "assy_hinban",
                   "格納位置_14", "格納時間_14", "排出時間_14", "不良カテゴリ_16",
                   "格納位置_33", "格納時間(YYMM)_33", "排出時間(YYMM)_33", "自動外観結果（1＝OK,2=NG)_33",
                   "シミ面積Aエリア2_33", "シミ面積Bエリア2_33"]
    
    col_rename  = {"格納位置_14": "storage_loc_14",
                   "格納時間_14": "storage_start_14",
                   "排出時間_14": "storage_end_14",
                   "不良カテゴリ_16": "defective_cat_16",
                   "格納位置_33": "storage_loc_33",
                   "格納時間(YYMM)_33": "storage_start_33",
                   "排出時間(YYMM)_33": "storage_end_33",
                   "自動外観結果（1＝OK,2=NG)_33": "visual_inspection_33",
                   "シミ面積Aエリア2_33": "blot_areaA_33", 
                   "シミ面積Bエリア2_33": "blot_areaB_33"}
    
    col_process = col_others + col_out
    
    #指定工程Noのデータ読み込み
    with codecs.open(os.path.join(r_path, csv_file), "r", "Shift-JIS", "ignore") as f:
        reader = pd.read_table(f, delimiter=",", low_memory=False, usecols=col_process, chunksize=1000)
        df = pd.concat((r for r in reader), ignore_index=True)

    #列名をリネーム
    df = df.rename(columns=col_rename)
    
    #日付：オブジェクト　→　datetimeへ変換
    for i in pro_no:
        df["out_put_time_%s" %i] = pd.to_datetime(df["out_put_time_%s" %i])
    for i in [14, 33]:
        df["storage_start_%s" %i] = pd.to_datetime(df["storage_start_%s" %i])
        df["storage_end_%s" %i]   = pd.to_datetime(df["storage_end_%s" %i])
    df["out_put_time_16"] = pd.to_datetime(df["out_put_time_16"])
    
    #工程間の時間　秒
    df["ti_16_to_32[s]"] = df[["out_put_time_16", "out_put_time_32"]].apply(CalcDeltaSeconds, axis=1)
    for (i, j) in zip(pro_no[:-1], pro_no[1:]):
        df["ti_%s_to_%s[s]" %(i, j)] = df[["out_put_time_%s" %(i), "out_put_time_%s" %(j)]].apply(CalcDeltaSeconds, axis=1)
    
    #マンション在宅時間　秒
    for i in [14, 33]:
        df["cure_time_%s[s]" %i] = df[["storage_start_%s" %i, "storage_end_%s" %i]].apply(CalcDeltaSeconds, axis=1)
    
    #先にドロップすると，色んな行のNAにヒットしてデータがほぼなくなるので注意
    df = df.dropna()
    """
    #不良ファイルから，基板品番→基板ロット情報抽出
    df_sub = pd.read_csv(os.path.join("..", "SA_DefectiveRatio.csv"))
    df_sub_lot = pd.DataFrame(index=[], columns=["Sub%s" %i for i in [1,2,3,4]])
    
    for i in np.arange(df.shape[0]):
        sub_lot = df_sub[df_sub["Lot"].astype(str) == df.loc[i, "kiban_lot"]]
        print(sub_lot)
        if len(sub_lot) == 0:
            sub_lot = [np.nan, np.nan. np.nan, np.nan]
        else:
            sub_lot = sub_lot.loc[:, ["Sub%s" %i for i in [1,2,3,4]]]
        df_sub_lot = pd.concat([df_sub_lot, sub_lot])
    
    df = pd.concat([df, df_sub_lot], axis=1)
    """
    #書き出し
    df.to_csv(os.path.join(w_path, "Assy_Sub_" + csv_file), index=False, encoding="utf-8")

In [48]:
#Assyゲル気泡
def CreateAssyBaseDataWithGel(csv_file):
    
    #工程番号
    pro_no = np.append(np.arange(16, 17), np.arange(32, 38))
    
    #読み込む列
    col_out     = ["out_put_time_%s"%i for i in pro_no]
    col_others  = ["sa_lot", "sa_seri", "sa_hinban", "kiban_lot", "film_lot", "assy_lot", "assy_hinban",
                   "基板高さ(L)_a_32", "基板高さ(L)_b_32", "基板高さ(R)_a_32", "基板高さ(R)_b_32", 
                   "格納位置_33", "格納時間(YYMM)_33", "排出時間(YYMM)_33", "自動外観結果（1＝OK,2=NG)_33",
                   "シミ面積Aエリア2_33", "シミ面積Bエリア2_33", "レーザ除去痕（1＝あり、2＝なし）_33",
                   "ゲル注入量（プランジャストローク）_a_35", "ゲル注入量（プランジャストローク）_b_35",
                   "基板高さ_a_35", "基板高さ_b_35", "硬化前ゲル厚_a_35", "硬化前ゲル厚_b_35", 
                   "格納位置_36", "格納時間(YYMM)_36", "排出時間(YYMM)_36", "不良カテゴリ_37",
                   "硬化後ゲル厚1_a_37", "硬化後ゲル厚1_b_37", "硬化後ゲル厚2_a_37", "硬化後ゲル厚2_b_37", 
                   "硬化後ゲル厚3_a_37", "硬化後ゲル厚3_b_37", "号機_37"]
    
    col_rename  = {"基板高さ(L)_a_32":"base_height_L_a_32",
                   "基板高さ(L)_b_32":"base_height_L_b_32",
                   "基板高さ(R)_a_32":"base_height_R_a_32",
                   "基板高さ(R)_b_32":"base_height_R_b_32",
                   "格納位置_33":"storage_loc_33",
                   "格納時間(YYMM)_33":"storage_start_33",
                   "排出時間(YYMM)_33":"storage_end_33",
                   "自動外観結果（1＝OK,2=NG)_33":"visual_inspection_33",
                   "シミ面積Aエリア2_33":"blot_areaA_33", 
                   "シミ面積Bエリア2_33":"blot_areaB_33", 
                   "レーザ除去痕（1＝あり、2＝なし）_33":"LASER_abl_mark_33",
                   "ゲル注入量（プランジャストローク）_a_35":"gel_plunger_stroke_a_35",
                   "ゲル注入量（プランジャストローク）_b_35":"gel_plunger_stroke_b_35",
                   "基板高さ_a_35":"base_height_a_35",
                   "基板高さ_b_35":"base_height_b_35",
                   "硬化前ゲル厚_a_35":"gel_thick_b4_a_35",
                   "硬化前ゲル厚_b_35":"gel_thick_b4_b_35",
                   "格納位置_36":"storage_loc_36",
                   "格納時間(YYMM)_36":"storage_start_36",
                   "排出時間(YYMM)_36":"storage_end_36",
                   "不良カテゴリ_37":"defective_cat_37",
                   "硬化後ゲル厚1_a_37":"gel_thick1_aft_a_37",
                   "硬化後ゲル厚1_b_37":"gel_thick1_aft_b_37",
                   "硬化後ゲル厚2_a_37":"gel_thick2_aft_a_37",
                   "硬化後ゲル厚2_b_37":"gel_thick2_aft_b_37",
                   "硬化後ゲル厚3_a_37":"gel_thick3_aft_a_37",
                   "硬化後ゲル厚3_b_37":"gel_thick3_aft_b_37",
                   "号機_37":"machine_no_37"}
    
    col_process = col_others + col_out
    
    #指定工程Noのデータ読み込み
    with codecs.open(os.path.join(r_path, csv_file), "r", "Shift-JIS", "ignore") as f:
        reader = pd.read_table(f, delimiter=",", low_memory=False, usecols=col_process, chunksize=1000)
        df = pd.concat((r for r in reader), ignore_index=True)

    #列名をリネーム
    df = df.rename(columns=col_rename)
    
    #日付：オブジェクト　→　datetimeへ変換
    for i in pro_no:
        df["out_put_time_%s" %i] = pd.to_datetime(df["out_put_time_%s" %i])
    
    df["storage_start_33"] = pd.to_datetime(df["storage_start_33"])
    df["storage_end_33"]   = pd.to_datetime(df["storage_end_33"])
    df["out_put_time_16"] = pd.to_datetime(df["out_put_time_16"])
    
    #工程間の時間　秒
    df["ti_16_to_32[s]"] = df[["out_put_time_16", "out_put_time_32"]].apply(CalcDeltaSeconds, axis=1)
    for (i, j) in zip(pro_no[:-1], pro_no[1:]):
        df["ti_%s_to_%s[s]" %(i, j)] = df[["out_put_time_%s" %(i), "out_put_time_%s" %(j)]].apply(CalcDeltaSeconds, axis=1)
    
    #マンション在宅時間　秒
    df["cure_time_33[s]"] = df[["storage_start_33", "storage_end_33"]].apply(CalcDeltaSeconds, axis=1)
    
    #ゲルに関する16進数を10進数に変換
#    for (drct, alph) in itertools.product(["L", "R"], ["a", "b"]):
#        df["base_height_%s_%s_32" %(drct, alph)] = df["base_height_%s_%s_32" %(drct, alph)].map(HEX2DN)
        
    for alph in ["a", "b"]:
        df["gel_plunger_stroke_%s_35" %alph] = df["gel_plunger_stroke_%s_35" %alph].map(HEX2DN)
        df["base_height_%s_35" %alph] = df["base_height_%s_35" %alph].map(HEX2DN)
        df["gel_thick_b4_%s_35" %alph] = df["base_height_%s_35" %alph].map(HEX2DN) / 100  #定義
    
    #先にドロップすると，色んな行のNAにヒットしてデータがほぼなくなるので注意
    df = df.dropna()

    #書き出し
    df.to_csv(os.path.join(w_path, "AssyWithGel_" + csv_file), index=False, encoding="utf-8")

In [14]:
#SA + Assy基板接着
def CreateAllBaseData(csv_file):
    
    #工程番号
    pro_no = np.append(np.arange(11, 17), np.arange(32, 34)) #31は当てにならないの
    
    #読み込む列
    #"chip_id","assy_lot", "assy_hinban",は16番でNGのものはNAになっているので，省いた
    col_out  = ["out_put_time_%s"%i for i in pro_no]
    col_others  = ["sa_lot", "sa_seri", "sa_hinban", "kiban_lot", "film_lot", "assy_lot", "assy_hinban",
                   "格納位置_14", "格納時間_14", "排出時間_14", "不良カテゴリ_16",
                   "格納位置_33", "格納時間(YYMM)_33", "排出時間(YYMM)_33", "自動外観結果（1＝OK,2=NG)_33",
                   "シミ面積Aエリア2_33", "シミ面積Bエリア2_33"]
    
    col_rename  = {"格納位置_14": "storage_loc_14",
                   "格納時間_14": "storage_start_14",
                   "排出時間_14": "storage_end_14",
                   "不良カテゴリ_16": "defective_cat_16",
                   "格納位置_33": "storage_loc_33",
                   "格納時間(YYMM)_33": "storage_start_33",
                   "排出時間(YYMM)_33": "storage_end_33",
                   "自動外観結果（1＝OK,2=NG)_33": "visual_inspection_33",
                   "シミ面積Aエリア2_33": "blot_areaA_33", 
                   "シミ面積Bエリア2_33": "blot_areaB_33"}
    
    col_process = col_others + col_out
    
    #指定工程Noのデータ読み込み
    with codecs.open(os.path.join(r_path, csv_file), "r", "Shift-JIS", "ignore") as f:
        reader = pd.read_table(f, delimiter=",", low_memory=False, usecols=col_process, chunksize=1000)
        df = pd.concat((r for r in reader), ignore_index=True)

    #列名をリネーム
    df = df.rename(columns=col_rename)
    
    #日付：オブジェクト　→　datetimeへ変換
    for i in pro_no:
        df["out_put_time_%s" %i] = pd.to_datetime(df["out_put_time_%s" %i])
    for i in [14, 33]:
        df["storage_start_%s" %i] = pd.to_datetime(df["storage_start_%s" %i])
        df["storage_end_%s" %i]   = pd.to_datetime(df["storage_end_%s" %i])
    df["out_put_time_16"] = pd.to_datetime(df["out_put_time_16"])
    
    #工程間の時間　秒
    for (i, j) in zip(pro_no[:-1], pro_no[1:]):
        df["ti_%s_to_%s[s]" %(i, j)] = df[["out_put_time_%s" %(i), "out_put_time_%s" %(j)]].apply(CalcDeltaSeconds, axis=1)
    
    #マンション在宅時間　秒
    for i in [14, 33]:
        df["cure_time_%s[s]" %i] = df[["storage_start_%s" %i, "storage_end_%s" %i]].apply(CalcDeltaSeconds, axis=1)
    
    #基板ロットが1セルに複数入っているのを分割
    df["kiban_lot"] = df["kiban_lot"].map(DecomposeStr)
    
    #先にドロップすると，色んな行のNAにヒットしてデータがほぼなくなるので注意
    df = df.dropna()
    df = df.reset_index()
    df = df.drop("index", axis=1)
    
    #不良ファイルから，基板品番→基板ロット情報抽出
    #定義＆読み込み
    sub_cols=["sub%s" %i for i in [1,2,3,4]]
    df_sub = pd.read_csv(os.path.join("..", "SA_DefectiveRatio.csv"))
    df_sub_lot = pd.DataFrame(index=[], columns=sub_cols)
    
    #1行ずつ，ヒットする基板Noを抽出
    for i in np.arange(df.shape[0]):
        
        sub_lot = df_sub[df_sub["Lot"].astype(str) == df.loc[i, "kiban_lot"]]

        if len(sub_lot) == 0:
            sub_ser = pd.Series([np.nan, np.nan, np.nan, np.nan], index=df_sub_lot.columns)
        else:
            sub_ser = sub_lot.loc[:, sub_cols]
        df_sub_lot = df_sub_lot.append(sub_ser, ignore_index=True)

    #大元のファイルと結合
    df = pd.concat([df, df_sub_lot], axis=1)
   
    #書き出し
    df.to_csv(os.path.join(w_path, "All_Sub_" + csv_file), index=False, encoding="utf-8")

In [15]:
#OK, NGファイルを結合
def CreateConcatCSV(csv_files):
    
    i = 0
    
    for csv_file in csv_files:
        reader = pd.read_csv(os.path.join(w_path, csv_file), low_memory=False, chunksize=2000)
        df = pd.concat((r for r in reader), ignore_index=True)
        
        if i == 0:
            df_res = pd.DataFrame(index=[], columns=df.columns)
            pro = csv_file.find("_")
            
        df_res = pd.concat([df_res, df])
        i += 1
        
    #df_res = df_res.dropna()
    df_res = df_res.sort_values("sa_lot")
    
    df_res.to_csv(os.path.join(w_path, "Concat_" + csv_file[:pro] + "_files.csv"), index=False)

■■■■■　MainProgram　■■■■■

In [16]:
#定義
r_path = os.path.join("..", "AssemblyData", "Original")
w_path = os.path.join("..", "AssemblyData", "Extract_Gel")

In [49]:
#オリジナルのファイルから，必要部を抜き出し
def GetExtractedData():
    csv_files = GetCSVFiles()
    
    for csv_file in csv_files:
        #CreateSABaseData(csv_file)
        #CreateAssyBaseData(csv_file)
        #CreateAllBaseData(csv_file)
        CreateAssyBaseDataWithGel(csv_file)
    
    #sa_files = GetSAExtractedFiles()
    #CreateConcatCSV(sa_files)
    
    #assy_files = GetAssyExtractedFiles()
    #CreateConcatCSV(assy_files)
    
    assy_with_gel_files = GetAssyWithGelExtractedFiles()
    CreateConcatCSV(assy_with_gel_files)
    
    #all_files = GetAllExtractedFiles()
    #CreateConcatCSV(all_files)

In [50]:
GetExtractedData()

■■■■■　確認　■■■■■

■■■■■　バックアップ　■■■■■

#基板ロットを入れない場合
def CreateSABaseData(csv_file):
    
    #工程番号
    pro_no = np.arange(11, 17)
    
    #読み込む列
    #"chip_id","assy_lot", "assy_hinban",は16番でNGのものはNAになっているので，省いた
    col_output  = ["out_put_time_%s"%i for i in pro_no]
    col_others  = ["sa_lot", "sa_seri", "sa_hinban", "kiban_lot", "film_lot",
                   "格納位置_14", "格納時間_14", "排出時間_14", "不良カテゴリ_16"]
    col_rename  = {"格納位置_14": "storage_loc_14",
                   "格納時間_14": "storage_start_14",
                   "排出時間_14": "storage_end_14",
                   "不良カテゴリ_16": "defective_cat_16"}
    col_process = col_others + col_output
    
    #指定工程Noのデータ読み込み
    with codecs.open(os.path.join(r_path, csv_file), "r", "Shift-JIS", "ignore") as f:
        reader = pd.read_table(f, delimiter=",", low_memory=False, usecols=col_process, chunksize=1000)
        df = pd.concat((r for r in reader), ignore_index=True)

    #列名をリネーム
    df = df.rename(columns=col_rename)
    
    #日付：オブジェクト　→　datetimeへ変換
    for i in pro_no:
        df["out_put_time_%s" %i] = pd.to_datetime(df["out_put_time_%s" %i])
        
    df["storage_start_14"] = pd.to_datetime(df["storage_start_14"])
    df["storage_end_14"]   = pd.to_datetime(df["storage_end_14"])
    
    #工程間の時間　秒
    for i in pro_no[:-1]:
        df["ti_%s_to_%s[s]" %(i, i+1)] = df[["out_put_time_%s" %(i), "out_put_time_%s" %(i+1)]].apply(CalcDeltaSeconds, axis=1)
    
    #for i in pro_no[:-1]:
    #    for j in pro_no[1:]:
    #        if i < j:
    #            df["ti_%s_%s[s]" %(i, j)] = (df["out_put_time_%s" %(j)] - df["out_put_time_%s" %(i)]).dt.seconds
    
    #マンション在宅時間　秒
    df["cure_time[s]"] = df[["storage_start_14", "storage_end_14"]].apply(CalcDeltaSeconds, axis=1)
    
    #先にドロップすると，色んな行のNAにヒットしてデータがほぼなくなるので注意
    df = df.dropna()
   
    #書き出し
    df.to_csv(os.path.join(w_path, "SA_" + csv_file), index=False, encoding="utf-8")

def CreateAssyBaseData(csv_file):
    
    #工程番号
    pro_no = np.arange(32, 34)
    
    #読み込む列
    #"chip_id","assy_lot", "assy_hinban",は16番でNGのものはNAになっているので，省いた
    col_out  = ["out_put_time_16"]+["out_put_time_%s"%i for i in pro_no]
    col_others  = ["sa_lot", "sa_seri", "sa_hinban", "kiban_lot", "film_lot", "assy_lot", "assy_hinban",
                   "格納位置_14", "格納時間_14", "排出時間_14", "不良カテゴリ_16",
                   "格納位置_33", "格納時間(YYMM)_33", "排出時間(YYMM)_33", "自動外観結果（1＝OK,2=NG)_33",
                   "シミ面積Aエリア2_33", "シミ面積Bエリア2_33"]
    
    col_rename  = {"格納位置_14": "storage_loc_14",
                   "格納時間_14": "storage_start_14",
                   "排出時間_14": "storage_end_14",
                   "不良カテゴリ_16": "defective_cat_16",
                   "格納位置_33": "storage_loc_33",
                   "格納時間(YYMM)_33": "storage_start_33",
                   "排出時間(YYMM)_33": "storage_end_33",
                   "自動外観結果（1＝OK,2=NG)_33": "visual_inspection_33",
                   "シミ面積Aエリア2_33": "blot_areaA_33", 
                   "シミ面積Bエリア2_33": "blot_areaB_33"}
    
    col_process = col_others + col_out
    
    #指定工程Noのデータ読み込み
    with codecs.open(os.path.join(r_path, csv_file), "r", "Shift-JIS", "ignore") as f:
        reader = pd.read_table(f, delimiter=",", low_memory=False, usecols=col_process, chunksize=1000)
        df = pd.concat((r for r in reader), ignore_index=True)

    #列名をリネーム
    df = df.rename(columns=col_rename)
    
    #日付：オブジェクト　→　datetimeへ変換
    for i in pro_no:
        df["out_put_time_%s" %i] = pd.to_datetime(df["out_put_time_%s" %i])
    for i in [14, 33]:
        df["storage_start_%s" %i] = pd.to_datetime(df["storage_start_%s" %i])
        df["storage_end_%s" %i]   = pd.to_datetime(df["storage_end_%s" %i])
    df["out_put_time_16"] = pd.to_datetime(df["out_put_time_16"])
    
    #工程間の時間　秒
    df["ti_16_to_32[s]"] = df[["out_put_time_16", "out_put_time_32"]].apply(CalcDeltaSeconds, axis=1)
    for (i, j) in zip(pro_no[:-1], pro_no[1:]):
        df["ti_%s_to_%s[s]" %(i, j)] = df[["out_put_time_%s" %(i), "out_put_time_%s" %(j)]].apply(CalcDeltaSeconds, axis=1)
    
    #マンション在宅時間　秒
    for i in [14, 33]:
        df["cure_time_%s[s]" %i] = df[["storage_start_%s" %i, "storage_end_%s" %i]].apply(CalcDeltaSeconds, axis=1)
    
    #先にドロップすると，色んな行のNAにヒットしてデータがほぼなくなるので注意
    df = df.dropna()
    df = df.reset_index()
    df = df.drop("index", axis=1)
    
    #不良ファイルから，基板品番→基板ロット情報抽出
    #定義＆読み込み
    sub_cols=["sub%s" %i for i in [1,2,3,4]]
    df_sub = pd.read_csv(os.path.join("..", "SA_DefectiveRatio.csv"))
    df_sub_lot = pd.DataFrame(index=[], columns=sub_cols)

    #1行ずつ，ヒットする基板Noを抽出
    for i in np.arange(df.shape[0]):
        sub_lot = df_sub[df_sub["Lot"].astype(str) == df.loc[i, "kiban_lot"]]
            
        if len(sub_lot) == 0:
            sub_ser = pd.Series([np.nan, np.nan, np.nan, np.nan], index=df_sub_lot.columns)
        else:
            sub_ser = sub_lot.loc[:, sub_cols]
        df_sub_lot = df_sub_lot.append(sub_ser, ignore_index=True)

    #大元のファイルと結合
    df = pd.concat([df, df_sub_lot], axis=1)
    
    #書き出し
    df.to_csv(os.path.join(w_path, "Assy_Sub_" + csv_file), index=False, encoding="utf-8")